<a href="https://colab.research.google.com/github/sreedattu07/sreedattu07/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [15]:
!kaggle datasets download imneonizer/normal-vs-camouflage-clothes

100% 1.82G/1.82G [01:19<00:00, 28.1MB/s]
100% 1.82G/1.82G [01:19<00:00, 24.7MB/s]


In [16]:
!unzip -qq normal-vs-camouflage-clothes.zip

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import shutil
import os

In [18]:
def plt_imshow(title, image):
	# convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

In [19]:
class Config:
    # initialize the path to the *original* input directory of images
    ORIG_INPUT_DATASET = "/content/8k_normal_vs_camouflage_clothes_images"

    # initialize the base path to the *new* directory that will contain
    # our images after computing the training and testing split
    BASE_PATH = "/content/sample_data"

    # derive the training, validation, and testing directories
    TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
    VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
    TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

    # define the amount of data that will be used training
    TRAIN_SPLIT = 0.75

    # the amount of validation data will be a percentage of the
    # *training* data
    VAL_SPLIT = 0.1

    # define the names of the classes
    CLASSES = ["camouflage_clothes", "normal_clothes"]

    # initialize the initial learning rate, batch size, and number of
    # epochs to train for
    INIT_LR = 1e-4
    BS = 32
    NUM_EPOCHS = 3

    # define the path to the serialized output model after training
    MODEL_PATH = "camo_detector.model"

# instantiate a Config object
config = Config()

In [20]:
# grab the paths to all input images in the original input directory
# and shuffle them
imagePaths = list(paths.list_images(config.ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

# compute the training and testing split
i = int(len(imagePaths) * config.TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

# we'll be using part of the training data for validation
i = int(len(trainPaths) * config.VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

# define the datasets that we'll be building
datasets = [
	("training", trainPaths, config.TRAIN_PATH),
	("validation", valPaths, config.VAL_PATH),
	("testing", testPaths, config.TEST_PATH)
]

In [21]:
for (dType, imagePaths, baseOutput) in datasets:
	# show which data split we are creating
	print("[INFO] building '{}' split".format(dType))

	# if the output base output directory does not exist, create it
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)

	# loop over the input image paths
	for inputPath in imagePaths:
		# extract the filename of the input image along with its
		# corresponding class label
		filename = inputPath.split(os.path.sep)[-1]
		label = inputPath.split(os.path.sep)[-2]

		# build the path to the label directory
		labelPath = os.path.sep.join([baseOutput, label])

		# if the label output directory does not exist, create it
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)

		# construct the path to the destination image and then copy
		# the image itself
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] 'creating /content/sample_data/training/normal_clothes' directory
[INFO] 'creating /content/sample_data/training/camouflage_clothes' directory
[INFO] building 'validation' split
[INFO] 'creating /content/sample_data/validation/camouflage_clothes' directory
[INFO] 'creating /content/sample_data/validation/normal_clothes' directory
[INFO] building 'testing' split
[INFO] 'creating /content/sample_data/testing/normal_clothes' directory
[INFO] 'creating /content/sample_data/testing/camouflage_clothes' directory


In [22]:
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--plot", type=str, default="plot.png",
#	help="path to output loss/accuracy plot")
#args = vars(ap.parse_args())

# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"plot": "plot.png"
}

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))

In [23]:
# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=25,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [24]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

Found 10731 images belonging to 2 classes.
Found 1192 images belonging to 2 classes.
Found 3975 images belonging to 2 classes.


In [25]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

[INFO] preparing model...


In [26]:
# compile the model
import tensorflow
opt = tensorflow.keras.optimizers.legacy.Adam(learning_rate=config.INIT_LR, decay=config.INIT_LR / config.NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the model
print("[INFO] training model...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // config.BS,
	validation_data=valGen,
	validation_steps=totalVal // config.BS,
	epochs=config.NUM_EPOCHS)

[INFO] training model...


<ipython-input-26-860bdb10b55e>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(


Epoch 1/3
335/335 [==============================] - 2560s 8s/step - loss: 0.2041 - accuracy: 0.9244 - val_loss: 0.0847 - val_accuracy: 0.9704
Epoch 2/3
335/335 [==============================] - 2550s 8s/step - loss: 0.0876 - accuracy: 0.9718 - val_loss: 0.0668 - val_accuracy: 0.9780
Epoch 3/3
335/335 [==============================] - 2551s 8s/step - loss: 0.0721 - accuracy: 0.9760 - val_loss: 0.0656 - val_accuracy: 0.9764


In [27]:
# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(testGen,
	steps=(totalTest // config.BS) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

# serialize the model to disk
print("[INFO] saving model...")
model.save(config.MODEL_PATH, save_format="h5")

[INFO] evaluating network...
125/125 [==============================] - 806s 6s/step
                    precision    recall  f1-score   support

camouflage_clothes       0.99      0.97      0.98      2007
    normal_clothes       0.97      0.99      0.98      1968

          accuracy                           0.98      3975
         macro avg       0.98      0.98      0.98      3975
      weighted avg       0.98      0.98      0.98      3975

[INFO] saving model...


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
import cv2

def predict_camouflage(image_path, model):
    # Load the image
    img = cv2.imread(image_path)

    # Resize the image to the expected shape
    img = cv2.resize(img, (224, 224))

    # Preprocess the image
    img_array = np.expand_dims(img, axis=0)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)

    # Predict the camouflage
    prediction = model.predict(img_array)
    return prediction[0][0]

# Example usage
result = predict_camouflage('/content/8k_normal_vs_camouflage_clothes_images/normal_clothes/0001.jpg',model)
if result >= 0.5:
     print("The image contains camouflage clothing.")
else:
     print("The image does not contain camouflage clothing.")


1/1 [==============================] - 0s 289ms/step
The image does not contain camouflage clothing.
